## Part 1

Importing Data + Creating helper functions

In [1]:
import json

with open('purchase_orders.json', 'r') as file:
    data = json.load(file)
    
with open('penalty_rates.json', 'r') as file:
    penalty_rates_raw = json.load(file)

In [2]:
def sum_units_and_amounts(data):
    """
    takes in a list of dicts
    returns aggregated amounts and units for each customer
    
    
    {target: {
        units: 1000
        amounts: 10000
    }}
    
    """
    
    aggregated_data = {}
    for entry in data:
        customer = entry['Customer']
        order_number = entry['Order Number']
        units = entry['Units']
        amount = float(entry['Amount'][1:])
        
        if customer in aggregated_data:
            aggregated_data[customer]['units'] += units
            aggregated_data[customer]['amount'] += amount
        else:
            aggregated_data[customer] = {'units': 0, 'amount': 0}
        
    return aggregated_data


def sum_everyone(data):
    aggregated_data = {'units': 0, 'amount': 0}
    for entry in data:
        customer = entry['Customer']
        order_number = entry['Order Number']
        units = entry['Units']
        amount = float(entry['Amount'][1:])
        
        aggregated_data['units'] += units
        aggregated_data['amount'] += amount
        
    return aggregated_data
    
    
def organize_penalties(penalty_rates):
    mapping = {}
    for entry in penalty_rates:
        customer = entry['Customer']
        fill_rate_target = entry['Fill Rate Target']
        penalty = entry['Penalty']


        mapping[customer] = {'fill_rate_target': 0, 'penalty': 0}
        mapping[customer]['fill_rate_target'] = float(fill_rate_target.strip('%'))/100
        mapping[customer]['penalty'] = float(penalty.strip('%')) / 100
    return  mapping


# Part 2
def get_max_penalty(order_data, penalty_rates):

    # aggregated_customer_data[entry]['units'] * penalty_rates[entry]['fill_rate_target'] 
    sum_penalties = 0
    # per unit penalty
    for order in order_data:
        
        units_missing = (order['Units'] * penalty_rates[order['Customer']]['fill_rate_target'])
        penalty_rate = (penalty_rates[order['Customer']]['penalty'])
        per_unit_cost = (float(order['Amount'][1:]))/order['Units']
        
        sum_penalties += units_missing * penalty_rate * per_unit_cost

    return sum_penalties


# Part 3

def optimize_penalty(data, penalty_rates):
    # to minimize the penalty i think a good starting point is to fulfill as many high unit cost orders as we can, and then partially fill others
    penalties = 0

    units_left = 3459
    for order in data:
        
        units_fulfilled = min((order['Units'] * penalty_rates[order['Customer']]['fill_rate_target']), units_left)
        units_left -= units_fulfilled
        
        # print(units_fulfilled, order['Units'] * penalty_rates[order['Customer']]['fill_rate_target'] - units_fulfilled)
        
        penalty_rate = (penalty_rates[order['Customer']]['penalty'])
        per_unit_cost = (float(order['Amount'][1:]))/order['Units']
        
        penalties += (order['Units'] * penalty_rates[order['Customer']]['fill_rate_target'] - units_fulfilled) * penalty_rate * per_unit_cost
        # print(penalties)
    return penalties

    

In [3]:
print(sum_units_and_amounts(data))

{'Target': {'units': 2008, 'amount': 2168.0}, 'Walmart': {'units': 1875, 'amount': 2175.0}, 'CVS': {'units': 1890, 'amount': 1918.0}}


## Part 1: Sum all units and amounts

In [4]:
print(sum_everyone(data))

{'units': 6076, 'amount': 6673.0}


In [5]:
aggregated_customer_data = sum_units_and_amounts(data)

penalty_rates = organize_penalties(penalty_rates_raw)

penalty_rates

{'Target': {'fill_rate_target': 0.9, 'penalty': 0.01},
 'Walmart': {'fill_rate_target': 0.95, 'penalty': 0.02},
 'CVS': {'fill_rate_target': 0.9, 'penalty': 0.01}}

## Part 2

Write a function to determine the penalty if we ship nothing. Ie, we incur the maximum
penalty possible.

In [6]:
print(get_max_penalty(data, penalty_rates))

82.90700000000002


In [10]:
penalties = []

# aggregated_customer_data[entry]['units'] * penalty_rates[entry]['fill_rate_target'] 
sum_penalties = 0
# per unit penalty
for order in data:
    
    units_missing = (order['Units'] * penalty_rates[order['Customer']]['fill_rate_target'])
    penalty_rate = (penalty_rates[order['Customer']]['penalty'])
    per_unit_cost = (float(order['Amount'][1:]))/order['Units']
    
    penalties.append(units_missing * penalty_rate * per_unit_cost)
    
    sum_penalties += units_missing * penalty_rate * per_unit_cost

    

sum_penalties

82.90700000000002

## Part 3: How can penalty be minimized best?

#### Idea 1: fulfill as much as we can (up to fulfillment target rate) on the highest unit cost orders first

#### Idea 2: alternatively, we can optimize based on the highest penalty orders first.

#### I'll experiment with both to see which one is better

## Optimize penalty cost based on highest unit cost orders

In [7]:
# nifty trick i learned from leetcode to sort a dictionary in reverse order based on a key

sort_data_by_cost = sorted(data, key= lambda x: (float(x['Amount'][1:]))/x['Units'], reverse=True)
print(sort_data_by_cost)

[{'Customer': 'Walmart', 'Order Number': 127, 'Units': 104, 'Amount': '$304.00', 'Date': '1/1'}, {'Customer': 'CVS', 'Order Number': 125, 'Units': 102, 'Amount': '$172.00', 'Date': '1/1'}, {'Customer': 'Walmart', 'Order Number': 169, 'Units': 146, 'Amount': '$246.00', 'Date': '1/2'}, {'Customer': 'Target', 'Order Number': 171, 'Units': 148, 'Amount': '$248.00', 'Date': '1/2'}, {'Customer': 'Target', 'Order Number': 129, 'Units': 106, 'Amount': '$156.00', 'Date': '1/1'}, {'Customer': 'Target', 'Order Number': 123, 'Units': 100, 'Amount': '$130.00', 'Date': '1/1'}, {'Customer': 'CVS', 'Order Number': 128, 'Units': 105, 'Amount': '$133.00', 'Date': '1/1'}, {'Customer': 'Target', 'Order Number': 126, 'Units': 103, 'Amount': '$113.00', 'Date': '1/1'}, {'Customer': 'Walmart', 'Order Number': 124, 'Units': 101, 'Amount': '$110.00', 'Date': '1/1'}, {'Customer': 'Walmart', 'Order Number': 130, 'Units': 107, 'Amount': '$107.00', 'Date': '1/1'}, {'Customer': 'CVS', 'Order Number': 131, 'Units': 1

In [8]:
print(optimize_penalty(sort_data_by_cost, penalty_rates))

27.589499999999987


## Optimize Penalty based on Highest Penalty Cost

In [13]:
sort_data_by_penalty = [x for _,x in sorted(zip(penalties,data), key=lambda x: x[0], reverse=True)]

print(sort_data_by_penalty)
print(sort_data_by_cost == sort_data_by_penalty) # my intuition said that these would be similar (or identical), but i guess not!

[{'Customer': 'Walmart', 'Order Number': 127, 'Units': 104, 'Amount': '$304.00', 'Date': '1/1'}, {'Customer': 'Walmart', 'Order Number': 169, 'Units': 146, 'Amount': '$246.00', 'Date': '1/2'}, {'Customer': 'Walmart', 'Order Number': 166, 'Units': 143, 'Amount': '$143.00', 'Date': '1/2'}, {'Customer': 'Walmart', 'Order Number': 163, 'Units': 140, 'Amount': '$140.00', 'Date': '1/2'}, {'Customer': 'Walmart', 'Order Number': 160, 'Units': 137, 'Amount': '$137.00', 'Date': '1/2'}, {'Customer': 'Walmart', 'Order Number': 157, 'Units': 134, 'Amount': '$134.00', 'Date': '1/2'}, {'Customer': 'Walmart', 'Order Number': 154, 'Units': 131, 'Amount': '$131.00', 'Date': '1/2'}, {'Customer': 'Walmart', 'Order Number': 151, 'Units': 128, 'Amount': '$128.00', 'Date': '1/2'}, {'Customer': 'Walmart', 'Order Number': 148, 'Units': 125, 'Amount': '$125.00', 'Date': '1/2'}, {'Customer': 'Walmart', 'Order Number': 145, 'Units': 122, 'Amount': '$122.00', 'Date': '1/2'}, {'Customer': 'Walmart', 'Order Number':

In [14]:
print(optimize_penalty(sort_data_by_penalty, penalty_rates))

21.442000000000004


## Optimize based on given Order, lets see how much better these approaches are than just optimizing as we get them.

In [15]:
print(optimize_penalty(data, penalty_rates))

30.531999999999996


It looks like compared to optimizing in the given order, optimizing by penalty reduces the overall penalty by nearly 30%.